In [133]:
import requests
import json
import pandas as pd
from datetime import datetime
import time
from time import sleep
from itertools import chain
import sqlite3
import pytz

tz = pytz.timezone('America/Sao_Paulo')
date = datetime.now(tz=tz).strftime('%Y%m%d')

In [134]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)

    return conn

conn = create_connection(r'C:\Users\shigu\Desktop\dataeng_lendico\sqlite\sqlite.db')
cur = conn.cursor()
cur.close()
conn.close()

In [135]:
queue_url = r"https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
mastery_url = r"https://br1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/"

path_daily = r'./masters_daily/'
path_ids = r'./masters_ids/'

In [154]:
def get_urldata(url,summonerId=''):
    '''
    
    Get data from url based on queue_url or mastery_url (5x5 or summonerId)
       - summonerId: default is "", set a summonerId if data is from a specific summoner
       
    '''
    try:
        url = url + summonerId

        headers = {
          'X-Riot-Token': 'RGAPI-bbe6d76b-5f4f-4629-99fc-7e09f75b4090'
        }

        resp = requests.request("GET", url, headers=headers)
        data = json.loads(resp.text)

        if resp.status_code != 200:
            sleep(5)
            data = get_urldata(summonerId)
        
        return data
    except Exception as e:
        raise(e)

In [155]:
def export_rdata(path,date,data,summonerId='league'):
    '''
    
    Export data to .json files for 5x5 queue or summoner data(s).
    Will only return data if summonerId is not "league"
     - summonerId: default is "league", summonerId can be specified.
     - Specify a path for which type of data is being exported as .json.
    
    '''
    try:
        with open(path+f'masters_rankedsolo_{date}_{summonerId}.json','w+') as datafile:
            json.dump(data,datafile)
            datafile.close()
        if summonerId != 'league':
            return data
        else:
            return 0
    except Exception as e:
        print(e)

In [138]:
# Get 5x5 queue data
queue_data = get_urldata(queue_url)

# Export 5x5 queue data into .json file
export_rdata(path_daily,date,queue_data)

# Read queue data into dataframe
queue_data = pd.read_json(json.dumps(queue_data))  

# Exploded dict column values into columns
exploded_data = pd.DataFrame(queue_data['entries'].values.tolist(), index=queue_data.index)

# Concatenate exploded data into queue_data to get tabular denormalized format
tabular_data = pd.concat([queue_data,exploded_data],axis=1)
tabular_data.drop(columns='entries',inplace=True)

### Visualization plotly

In [139]:
# List of only few summonerIds
test_data =tabular_data['summonerId'][0:5]

In [145]:
# Get and store summonerId(s) data(s)
merged = [export_rdata(path_ids,date,get_urldata(mastery_url,_id),_id) for _index,_id in enumerate(test_data)]

In [147]:
# Transform list of dict into dataframe
mastery_data = pd.DataFrame(list(chain.from_iterable(merged)))

In [157]:
mastery_data.to_csv('pbitest.csv')

In [68]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px

mastery_data = mastery_data.set_index('summonerId')
mastery_data.sort_values(by=['championPoints','summonerId'], ascending=False,inplace=True)
mastery_data.championLevel = mastery_data.championLevel.astype(str)

def f(summonerId=mastery_data.index[0],n_champions='20'):
    fig = px.bar(mastery_data.loc[summonerId].head(int(n_champions)),
              x='championId', y='championPoints',
              title=f'Top {n_champions} Champion(s) Points per summonerId',color='championLevel'
             )
    
    fig.update_xaxes(type='category')
    fig.show()
    
    
fig = interact(f, summonerId=mastery_data.index,m_champions='')

interactive(children=(Dropdown(description='summonerId', options=('DfxYS3lzWoyLfurRQDECyWzNdd3Hr_Fu2gGOMidH7o6…

### Approach #1 -> Spark

data = spark.read.json(f'masters_rankedsolo_{date}.json')
data.printSchema()
- Transform json data into a tabular data

exploded_data = data.select('leagueId','name','queue','tier',explode('entries'))

exploded_data = exploded_data.select("col.*","*").drop('col').toPandas()